# SA-DISCnet Statistics: Exercises (Fun version)


Import python modules

In [4]:

import corner
import emcee
import math
import mpmath
import numpy as np
import pdb
import matplotlib.pyplot as plt
import scipy
import scipy.special
from scipy import optimize

# Q1 Maximum likelihood and Bayes example: exponential distribution

(a) Suppose a set of events has a distribution of times taken for the events, and you are trying to determine the mean time for this distribution. The distribution has $p(t) \propto e^{-t/\tau}$. You measure $N$ event times $t_i$. If there is no limit on times that can be measured, the PDF for measured separation time $t$ is $$p(t) = \frac{e^{-t/\tau}}{\tau}.$$ If you like, you can integrate this to show that it is normalised to 1.

Using the maximum likelihood method it is possible to show that estimates of $\tau$ and its variance are
given by  $$\widehat\tau = \frac{1}{N} \sum_i t_i$$ and $$V(\tau) = \frac{\widehat{\tau}^2}{N}.$$


Now suppose that you cannot measure any times longer than $T$. The truncated
PDF (normalised to 1) is then $$p(t) =\frac{e^{-t/\tau}}{\int_0^T e^{-t/\tau} dt} =\frac{1}{\tau}e^{-t/\tau}(1-e^{-T/\tau})^{-1}.$$ By differentiating the log likelihood $l$ with respect to  $\tau$ and setting it to zero, one can show that the maximum-likelihood estimate of $\tau$ is given by $$\widehat{\tau }=\frac{1}{N}\sum t_{i}+ \frac{Te^{-T/\widehat{\tau }}}{\left( 1-e^{-T/\widehat{\tau }}\right) }. \,\,\,(*)$$




(b) Below you'll see exp_trunc(), which generates $N$ = 1000 times from the original PDF, 
with  $\tau$= 10 s with a maximum of $T$ = 15 s. It plots a histogram of generated times. Work together to understand the code, and try different values of the parameters. You could also explore implementing different PDFs, using the transformation method. For instance, one can generate exponentially distributed
  random numbers $t$ from a uniform distribution $x$:
\begin{align*}
  x &= \int_0^t dt \frac{1}{\tau} e^{-t/\tau} = \left[-e^{-t/\tau}\right]_0^t
      = 1 - e^{-t/\tau}\\
  t &= -\tau \ln(1-x).
\end{align*}
You could dream up alternative distributions and see what happens. More on this in question 4 below.


(c) exp_trunc also implements equation (*) to estimate $\widehat\tau$ from the generated data. Try to understand how the code is doing this (and experiment with changing the code for this estimator). Is this consistent with the true
value of $\tau$, given the simplified estimate of variance on the estimate?


(d) Suppose that a previous experiment has estimated the mean time to be $\widehat\tau=\tau_p\pm\sigma$. This can act as a prior. It is possible to write down the posterior  by multiplying the likelihood from the new observations by the prior. Then one can differentiate the log likelihood, set it to zero, and hence find a maximum likelihood estimate for $\tau$.
This is found to be 
\begin{equation}
\widehat{\tau }=\frac{1}{N}\sum t_{i}+ \frac{Te^{-T/\widehat{%
      \tau }}}{\left( 1-e^{-T/\widehat{\tau }}\right) } -
\frac{1}{N} \frac{(\tau - \tau_P)\tau^2}{\sigma^2}.
\end{equation}


What maximum likelihood values do you obtain for $\widehat\tau$ for prior estimate $\tau_p$ = 8 s, $\sigma$ = 1 s,
and with computer generated data with $\tau$ = 10 s, $T$ = 15 s for different sample sizes $N$ =
10; 100; 1000 and 10000? Comment on your results. Try different values of the parameters, and try altering the code.




In [9]:
def exp_trunc(nran=1000, tau=10, T=15, tau_p=8, sigma=1):
    """Truncated exponential distribution."""
    t = np.random.exponential(tau, nran)
    t = t[t < T]
    tt = -tau*np.log(1 - np.random.random(nran))
    tt = tt[tt < T]
    nuse = len(t)
    plt.clf()
    plt.hist(t, 15)
#    plt.hist(tt, 15)
    plt.xlabel('t [s]')
    plt.ylabel('Frequency')
    plt.show()
    tmean = t.mean()

    def fn(te):
        return tmean + T*math.exp(-T/te) / (1 - math.exp(-T/te)) - te

    def fn_prior(te):
        return (tmean + T*math.exp(-T/te) / (1 - math.exp(-T/te))
                - (te - tau_p)*te**2 / (nuse*sigma**2) - te)

    res = scipy.optimize.fsolve(fn, tmean)
    print('tmean = {:4.1f}, maxL solution = {:4.1f}, est std = {:4.1f}'.format(
            tmean, res[0], res[0]/math.sqrt(nuse)))
    res = scipy.optimize.fsolve(fn_prior, tmean)
    print('With prior maxL solution = {:4.1f}, est std = {:4.1f}'.format(
            res[0], res[0]/math.sqrt(nuse)))


In [11]:
exp_trunc()

tmean =  5.5, maxL solution =  9.2, est std =  0.3
With prior maxL solution =  8.8, est std =  0.3


In [7]:
# Try calling exp_trunc with different values of N.  
# For low N, you should find that the estimate is pulled down by the prior, 
# but as N increases, the estimate should tend to the 'true' value.
exp_trunc(100)

tmean =  5.8, maxL solution = 10.4, est std =  1.2
With prior maxL solution =  8.4, est std =  0.9


## Q2 Bayes’ theorem example: constrained measurements

A small quantity of powder is being weighed on a balance which gives a reading of $x\pm\sigma$ g.
Assuming a uniform prior on the true mass $X$ for any positive value, $p(X > 0)$ = const, zero
otherwise, we can show that the posterior likelihood for the true mass is given by
$$p(X|x) = \sqrt{\frac{2}{\pi}}\frac{1}{\sigma}\frac{e^{-(x-X)^2/2\sigma^2}}{\mathrm{erfc}(-x/\sqrt{2}\sigma)} $$
for X > 0, zero otherwise. The argument is given below - try to follow it. (Don't worry if you feel that you'd never have thought of this - we gradually learn these tricks over years and years!)


For measurement $x$ and theory $X$, Bayes' theorem states
  \begin{equation}
    p(X|x) = \frac{p(x|X) p(X)}{p(x)}.
  \end{equation}

  For Gaussian measurement errors and a constant prior for positive $X$,
  we have:
  \begin{align*}
    p(x|X) &= \frac{e^{-(x-X)^2/2\sigma^2}}{\sigma \sqrt{2 \pi}},\\
    p(X) &= \mbox{ const if } X > 0, \mbox{ zero otherwise},\\
    p(x) &= \int_{-\infty}^\infty p(x|X) p(X) dX = \int_0^\infty p(x|X) dX.
  \end{align*}
  We thus have
\begin{equation}
p\left( X|x\right) =\frac{e^{-\left( x-X\right) ^{2}/2\sigma ^{2}}}
{\int_{0}^{\infty }e^{-( x-X) ^{2}/2\sigma^{2}}dX}
\mbox{ for $X > 0$, zero otherwise.}
\end{equation}

Now, substituting $y = (x - X)/\sqrt{2} \sigma$,
\begin{equation}
\int_{0}^{\infty }e^{-( x-X) ^{2}/2\sigma^{2}}dX =
\sqrt{2} \sigma \int_{-x/\sqrt{2} \sigma}^\infty e^{-y^2} dy =
\sqrt{2} \sigma \frac{\sqrt{\pi}}{2} {\rm erfc} (-x/\sqrt{2} \sigma),
\end{equation}
where
\begin{equation}
{\rm erfc}(x) = \frac{2}{\sqrt{\pi}} \int_x^\infty e^{-t^2} dt.
\end{equation}

Thus
\begin{equation}
p(X|x) =\sqrt{\frac{2}{\pi}} \frac{1}{\sigma} \frac{e^{-\left( x-X\right) ^{2}/2\sigma ^{2}}}
{{\rm erfc} (-x/\sqrt{2} \sigma)}
\mbox{ for $X > 0$, zero otherwise.}
\end{equation}

Bayes() below makes plots of $p(X|x)$. Try to understand how it works. Plot for $X$ between 0 and 1 g for $\sigma$ = 0.2 g and for mass readings $x$ = -0.3, -0.1, 0.1, 0.3 g
(plot a separate line for each of the four readings).

Try different readings, and experiment with what happens for different PDFs and priors.

In [12]:
def bayes(sigma=0.2, X=np.linspace(0.001, 1.0, 50)):
    """Bayesian limits on mass (Q3)"""

    def p(X, x, sigma):
        prob = np.zeros(len(X))
        idx = X > 0
        prob[idx] = (np.sqrt(2.0/math.pi) / sigma * np.exp(-(x - X[idx])**2/(2 * sigma**2))/
                     scipy.special.erfc(-x/(np.sqrt(2.0) * sigma)))
        return prob

    plt.clf()
    for x in (-0.3, -0.1, 0.1, 0.3):
        plt.plot(X, p(X, x, sigma), label='x = {}'.format(x))
    plt.xlabel(r'$X$')
    plt.ylabel(r'$P(X|x)$')
    plt.legend()
    plt.show()


In [17]:
bayes()

## Q3 MCMC example

In the first part of this question, we use a Monte Carlo program to simulate a sample drawn from a particular parameterised PDF. In the second part, we use the simulated data that we have generated, and do a maximum likelihood analysis to determine the parameters of the model. If
we are successful, our analysis should yield the initial input parameters - always a useful
check to perform when we are developing Monte Carlo routines!

(a) Event Generation.

i) The probability distribution function for detecting a mass $m$ in a particle physics experiment is 

$$p(m) = A \frac{\Gamma^2/4}{(m-m_0)^2+\Gamma^2/4}$$

where $m_0$ and $\Gamma$ are parameters of the PDF. By changing variables to $x = 2 (m - m_0)/\Gamma$, and integrating over $m$ to normalise the distribution, we can find that
$$A=\frac{2}{\pi \Gamma}.$$ (You might like to do this).


ii) Now, given a random number generator U (0, 1), we want to generate a sample from this distribution. The best way is the transformation method; it is possible to show that the transformation
$$m = m_0 + \frac{\Gamma}{2}\tan[(x-1/2)\pi]$$
􀀀
generates the distribution as required. [x is a random number generated from U (0, 1) ].

See the code in breit_wigner() below which does this. Understand how it's implemented.


iii) The code now generates 10,000 masses $m$ in this way, with $m_0$ = 784 MeV, 􀀀 $\Gamma$ = 12 MeV, and puts
them into a histogram with, say, 50 bins covering the mass range $m$ =760-810 MeV. Experiment with this part of the code, changing the histogram bin size, sample size or parameters $m_0$ and $\Gamma$.


(b) Now, imagine that we have carried out an experiment with a sample following this distribution. We shall use the Maximum Likelihood Method to find the best
estimate of $m_0$ and $\Gamma$). 

i) Since the normalised distribution is 
\begin{equation}
p(m) =\frac{2}{\pi \Gamma }\frac{\Gamma ^{2}/4}{\left( m-m_{0}\right)
^{2}+\Gamma ^{2}/4},
\end{equation}
make sure that you can see that the log likelihood is 
\begin{equation}
l=-N\ln 2\pi +\sum_{i=1}^{N}\ln \left\{ \frac{\Gamma }{\left(
m_{i}-m_{0}\right) ^{2}+\Gamma ^{2}/4}\right\} .
\end{equation}


ii) Use the code below to plot log likelihood vs. mass $m_0$ in the mass range 760-810 MeV, keeping $\Gamma$􀀀 constant at 12 MeV. Understand how the code is doing this, and try changing the range and $\Gamma$.


iii) Similarly use the code to plot log likelihood vs. $\Gamma$ 􀀀 in the range 8-16 MeV, keeping $m_0$ constant at 784
MeV. Try changing range and $m_0$.


iv) The variables $m_0$ and $\Gamma$􀀀 are correlated, and to find the global maximum you generally
need to iterate, taking it in turns to adjust each of the two variables and find the
peak in the other. (Remember, for the plots you drew above, you already "knew" the
answer for the complementary variable in each case, which is why they peaked in the
right place straight away).

The code uses Markov Chain Monte Carlo to explore the two-dimensional parameter space.
It can make plots of (i) the joint distribution in $m_0$ and $\Gamma$􀀀 and (ii) marginalised distributions in $m_0$ and $\Gamma$􀀀 separately.

See http://dfm.io/emcee/current/ for documentation.

Try changing the code in various ways to understand how it's working - e.g. change the number of walkers, the length of chains, the starting points. See if you can implement MCMC for a different log likelihood.


In [54]:
def breit_wigner(N=10000, m_true=784, gam_true=12, m_range=(783, 785),
                 gam_range=(11, 13), nchain=3, nstep=1000):
    """Simulation of Breit-Wigner distribution."""

    ln2pi = math.log(2*math.pi)

    def loglike(theta):
        m0 = theta[0]
        gamma = theta[1]
        mgrid, m0grid, gamgrid = np.meshgrid(m, m0, gamma)
        return np.squeeze(
            N*ln2pi +
            np.sum(np.log(gamgrid / ((mgrid-m0grid)**2 + gamgrid**2/4.0)),
                   axis=1))

    def lnprob(theta):
        m0 = math.exp(theta[0])
        gamma = math.exp(theta[1])
        ll = -N*ln2pi + np.sum(np.log(gamma / ((m-m0)**2 + gamma**2/4.0)))
#        print(theta, ll)
        return ll

    x = np.random.rand(N)
    m = m_true + 0.5*gam_true*np.tan((x-0.5)*math.pi)
    plt.clf()
    plt.hist(m, 50, (760, 810))
    plt.xlabel(r'$m$ (MeV)')
    plt.ylabel('Frequency')
    plt.show()

    mbins = np.linspace(m_range[0], m_range[1], 50)
    ll = loglike((mbins, gam_true))
    plt.clf()
    plt.plot(mbins, ll)
    plt.xlabel(r'$m_0$ (MeV)')
    plt.ylabel('log likelihood')
    plt.show()

    gambins = np.linspace(gam_range[0], gam_range[1], 50)
    ll = loglike((m_true, gambins))
    plt.clf()
    plt.plot(gambins, ll)
    plt.xlabel(r'$\Gamma$ (MeV)')
    plt.ylabel('log likelihood')
    plt.show()

    ndim = 2
    x0 = (7, 3)
    nwalkers = 10*ndim
    pos = np.tile(x0, (nwalkers, 1)) + 0.1*np.random.randn(nwalkers, ndim)
    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob)
    for ichain in range(nchain):
        sampler.reset()
        pos, prob, state = sampler.run_mcmc(pos, nstep)
        print("Mean acceptance fraction: {0:.3f}"
              .format(np.mean(sampler.acceptance_fraction)))
        try:
            print("Autocorrelation time:", sampler.get_autocorr_time(c=1))
        except:
            print("Unable to compute autocorrelation time")
        plt.clf()
        nrow = 2
        ncol = 1
        fig, axes = plt.subplots(nrow, ncol, num=1)
        fig.subplots_adjust(hspace=0, wspace=0.1)
        fig.text(0.5, 0.02, 'Step number', ha='center', va='center')
        fig.text(0.06, 0.7, r'$\ln m_0$', ha='center',
                 va='center', rotation='vertical')

        fig.text(0.06, 0.3, r'$\ln \Gamma$', ha='center',
                 va='center', rotation='vertical')
        for ipar in range(ndim):
            ax = axes[ipar]
            for iw in range(nwalkers):
                ax.plot(sampler.chain[iw, :, ipar])
#                if (iw == 0):
#                    print(sampler.chain[iw, :, ipar])
        plt.show()
    res = np.exp(np.array(np.percentile(sampler.flatchain, [50, 16, 84], axis=0)))
    print(res)

    samples = np.exp(sampler.chain.reshape((-1, ndim)))
    fig = corner.corner(samples, labels=["$m_0$", "$\Gamma$"],
                        truths=[m_true, gam_true])
    plt.show()

In [55]:
breit_wigner()

Mean acceptance fraction: 0.696
Autocorrelation time: [ 20.85805907  50.95038131]
Mean acceptance fraction: 0.724
Autocorrelation time: [ 21.21932431  29.78205562]
Mean acceptance fraction: 0.710
Autocorrelation time: [ 49.79526999  40.95463164]
[[ 783.94104018   11.88444898]
 [ 783.85331907   11.71240283]
 [ 784.02275372   12.06043692]]
[[  96.96090274   96.96090274   97.04349668 ...,   97.15531145
    97.15531145   97.15531145]
 [  23.40958227   23.40958227   23.44316534 ...,   23.49166816
    23.49166816   23.49166816]
 [ 401.6054858   401.6054858   401.71368118 ...,  401.80861061
   401.80861061  401.80861061]]


## Q5 Generating arbitrarily distributed random numbers

Utilising the uniform random number generator from Python, we can
write functions that will efficiently return $N$ random numbers corresponding to (a) a supplied
empirical distribution ${x_i, p_i(x_i)}$, (b) a user-supplied 1-d PDF $p(x)$ and (c) a user-supplied
2-d PDF $p(x, y)$.

This is implemented in ran_fun_test() below. Work together to understand the code, and experiment with different PDFs.

In [58]:
def ran_dist(x, p, nran):
    """Generate nran random points according to distribution p(x)"""

    if np.amin(p) < 0:
        print('ran_dist warning: pdf contains negative values!')
    cp = np.cumsum(p)
    y = (cp - cp[0]) / (cp[-1] - cp[0])
    r = np.random.random(nran)
    return np.interp(r, y, x)


def ran_fun(f, xmin, xmax, nran, args=None, nbin=1000):
    """Generate nran random points according to pdf f(x)"""

    x = np.linspace(xmin, xmax, nbin)
    if args:
        p = f(x, *args)
    else:
        p = f(x)
    return ran_dist(x, p, nran)


def ran_fun2(f, xmin, xmax, ymin, ymax, nran, args=(), nbin=1000, pplot=False):
    """Generate nran random points according to 2d pdf f(x,y)"""

    dx = float(xmax - xmin)/nbin
    dy = float(ymax - ymin)/nbin
    x = np.linspace(xmin + 0.5*dx, xmax - 0.5*dx, nbin)
    y = np.linspace(ymin + 0.5*dy, ymax - 0.5*dy, nbin)
    xv, yv = np.meshgrid(x, y)
    p = f(xv, yv, *args)
    if pplot:
        plt.clf()
        plt.imshow(p, aspect='auto', origin='lower',
                   extent=(xmin, xmax, ymin, ymax), interpolation='nearest')
        plt.colorbar()
    binno = range(nbin * nbin)
    bins = (ran_dist(binno, p, nran)).astype(int)
    j = bins // nbin
    i = bins % nbin
    xran = x[i]
    yran = y[j]

    # Add uniform random offsets to avoid quantization
    xoff = dx * (np.random.random(nran) - 0.5)
    yoff = dy * (np.random.random(nran) - 0.5)

    return xran + xoff, yran + yoff


def ran_fun_test():
    """Test ran_fun2"""

    def fun(x, y):
        return np.cos(x)**2 * np.sin(y)**2

    xr, yr = ran_fun2(fun, -5, 5, -5, 5, 10000, pplot=True)
    plt.scatter(xr, yr, 0.1, c='w')
    plt.show()


In [60]:
ran_fun_test()